# Model Training for Real Time Communication through AI for Specially Abled
### Loading the Dataset & Image Data Generation

In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras
!pip install tensorflow

In [3]:
import tensorflow as tf
print(tf.version.VERSION)

2.7.2


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Image Generators

In [5]:
# Training Datagen
train_datagen = ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
# Testing Datagen
test_datagen = ImageDataGenerator(rescale=1/255)

## Uploading Dataset

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='R4YKlqzncR6Z7GrtISxpdRxePyb1HWe3OFiOdyzYUTFi',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'asltospeech-donotdelete-pr-ckyuplj21lrffr'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


## Unzipping Dataset

In [7]:
from io import BytesIO
import zipfile
unzip= zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
pwd

'/home/wsuser/work'

In [9]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset/training_set')

In [10]:
# Training Dataset
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/training_set',target_size=(64,64), class_mode='categorical',batch_size=900)
# Testing Dataset
x_test=test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/test_set',target_size=(64,64), class_mode='categorical',batch_size=900)

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [11]:
print("Len x-train : ", len(x_train))
print("Len x-test : ", len(x_test))

Len x-train :  18
Len x-test :  3


In [12]:
# The Class Indices in Training Dataset
x_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

### Model Creation

In [14]:
# Importing Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense

In [15]:
# Creating Model
model=Sequential()

In [16]:
# Adding Layers
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Adding Hidden Layers
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))

# Adding Output Layer
model.add(Dense(9,activation='softmax'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 300)               9225900   
                                                                 
 dense_1 (Dense)             (None, 150)               45150     
                                                                 
 dense_2 (Dense)             (None, 9)                 1359      
                                                        

In [18]:
# Compiling the Model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
# Fitting the Model Generator
model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))

/tmp/wsuser/ipykernel_365/1042518445.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))


Epoch 1/10
18/18 [==============================] - 48s 3s/step - loss: 1.4103 - accuracy: 0.5365 - val_loss: 0.5789 - val_accuracy: 0.8662
Epoch 2/10
18/18 [==============================] - 47s 3s/step - loss: 0.3759 - accuracy: 0.8899 - val_loss: 0.2968 - val_accuracy: 0.9107
Epoch 3/10
18/18 [==============================] - 47s 3s/step - loss: 0.1654 - accuracy: 0.9538 - val_loss: 0.1994 - val_accuracy: 0.9529
Epoch 4/10
18/18 [==============================] - 48s 3s/step - loss: 0.0889 - accuracy: 0.9764 - val_loss: 0.1689 - val_accuracy: 0.9680
Epoch 5/10
18/18 [==============================] - 47s 3s/step - loss: 0.0525 - accuracy: 0.9858 - val_loss: 0.1462 - val_accuracy: 0.9724
Epoch 6/10
18/18 [==============================] - 47s 3s/step - loss: 0.0350 - accuracy: 0.9917 - val_loss: 0.1400 - val_accuracy: 0.9729
Epoch 7/10
18/18 [==============================] - 47s 3s/step - loss: 0.0243 - accuracy: 0.9941 - val_loss: 0.1516 - val_accuracy: 0.9756
Epoch 8/10
18/18 [==

### Saving the Model

In [20]:
model.save('ASL_Model.h5')

In [21]:
!tar -zcvf ASL_to_Speech_new.tgz ASL_Model.h5

ASL_Model.h5


In [22]:
ls -1

ASL_Model.h5
ASL_to_Speech_new.tgz
Dataset/


## IBM Watson Machine Learning

In [23]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 19.8 MB/s eta 0:00:01


In [24]:
from ibm_watson_machine_learning import APIClient
wml_credentials ={
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey" : "yHjaItlJwtgu_lAkmRcKhBy983oaArks8Jgv8qwWwdk9"
}
client =APIClient(wml_credentials)

In [25]:
client =APIClient(wml_credentials)

In [26]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [27]:
space_uid = guid_from_space_name(client, 'ASL_to_Speech')
print("Space UID: "+ space_uid)

Space UID: 4709c4ee-cb66-4736-820d-2707c282ef59


In [28]:
client.set.default_space(space_uid)

'SUCCESS'

In [29]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [30]:
software_spec_uid= client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [31]:
model_details = client.repository.store_model(model='ASL_to_Speech_new.tgz',meta_props={
     client.repository.ModelMetaNames.NAME:"CNN",
     client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
     client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid}
                                             )                                          
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [32]:
model_id

'fb42adf7-1c59-470c-b081-92da64bc6692'

In [33]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

## Model testing

In [34]:
from keras.models import load_model
from keras.preprocessing import image

In [35]:
model = load_model("ASL_Model.h5")

In [47]:
img = image.load_img(r'/home/wsuser/work/Dataset/test_set/C/1.png', target_size =(64,64))

In [48]:
import numpy as np
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)

In [49]:
pred=np.argmax(model.predict(x),axis=1)

In [50]:
pred[0]

2

In [51]:
index=['A','B','C','D','E','F','G','H','I']
print(index[pred[0]])

C
